# Solving electric field problems numerically

One type of problem you will encounter in electrostatics is one where you know the distribution of charge at every point in space (or every point where the sources exist) and you need to determine the electric field everywhere in space (or everywhere you are interested). This can be done using Coulomb's law,

$$ \vec{E} = \int_V d\tau\;\dfrac{1}{4\pi\varepsilon_0}\dfrac{dq}{|\mathfrak{\vec{r}}|^2}\hat{\mathfrak{r}}$$

This calculation might be difficult to perform for all locations that you are interested in, or it might be downright impossible to solve analytical as no anti-derivative might exist for the function. In this case, using numerical techniques makes the problem tractable.

## The concept of numerical integration 

Typically, when you perform an integral analytically, you are seeking the anti-derivative of the function that you are integrating. This is one productive way to think about an integral. 

Another productive way to think about integration is adding up small bits. You might think about this when you do a line integral, but this can also be a productive way of thinking about any kind of integral, in particular, numerical integration. To conceptualize a numerical integral where you are trying to determine the electric field it is important to follow these steps:

1. Divide the source into chunks ($dq$) and pick a starting chunk
2. Determine the separation vector ($\vec{\mathfrak{r}}$) between the chunk and the observation location
3. Compute the contribution to the total electric field from the chunk ($d\vec{E}$) - treating it as a point charge of size $dq$
4. Add this contribution to the "running" total of electric field
5. Repeat steps 2-5, which form the basis of numerical integration (it's the superposition of the chunks), until you have added up the contributions of all the chunks

In this problem, you will calculate the electric field of line charge numerically (first) at a point along the middle. We do this because you know the analytical function (and thus the value) that describes the electric field at this point, which allows you to check your answer.

## The ```while``` loop

In order to find the electric field, you will need to continue to perform certain calculations (steps 2-4 above) until you have done so for all the chunks. In python, we will often use the ```while``` loop to do such calculations. The ```while``` loop will perform a series of calculations in order until the loop condition is satisfied. Below is a simple example, which prints all the numbers from 0 to 10 in order.

In [ ]:
j = 0           ## Set the counter to 0

while j < 11:   ## Run the commands in the loop until j=11 (i.e., print up to number 10)
                ## Notice that the while loop ends with a colon
    
    print(j)    ## Print the value j; notice the commands in the while loop are idented over 4 spaces
                ## Jupyter automatically does the indenting if you use the colon
    
    j = j + 1   ## Increment (increase) the counter by 1 unit

The loop above prints the numbers 0 through 10. Each line in the ```while``` loop is executed in order until the loop condition ```while j < 11:``` is met (i.e., j becomes 11).

# Problems to Solve
## Determining the electric field of a charged rod numerically

You have already calculated (by hand) an approximate value for the charged rod, which demonstrated the procedure of a numerical integral. Now you will use Python to extend that calculation to more chunks to investigate how many chunks are needed to get near the analytical solution. 

The code below starts the calculation but is missing an essential piece of the physics needed for the calculation. The code will be described in detail below.

### Importing division and vpython

The two statements below import two libraries that you will use in this notebook. 

* The first line makes sure that "floating point" division occurs (this line is only needed for Python 2.7, but doesn't hurt Python 3.5). Basically, it makes sure that Python treats something like 3/2 as 1.5 and not 0!
* The second line imports the visualization tools of [vpython](http://vpython.org), so you can create rods, spheres, arrows, and others things easily. VPython will be used a lot in this course as it will help us visualize the fields and interactions we are interested in.

In [1]:
from __future__ import division    ## Import floating point division
from vpython import *              ## Import vpython for visualizations

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Set up constants and parameters

Numerically integrated problems are (typically) solved for a set of conditions; they are not often solved in the abstract (i.e., with a function that describes the solution for all possible scenarios). 

Below, we define $k=\dfrac{1}{4\pi\varepsilon_0}$, the total charge of the rod, $Q$, and the length of the rod, $L$. In addition, the number of chunks, $N$, and the charge of each chunk $dq$ are also defined.

The final defined quantity, ```EScale```, is a scale factor for redusing the size of the arrows that vpython produces so it can be seen reasonably well. After you get the code working, you can play with that scale factor to see its effect. 

In [2]:
k = 9e9           ## Permitivity of free space in SI units
Q = 0.1e-6        ## Total charge distributed over the rod in Coulombs
L = 1             ## Length of the rod in meters

N = 4             ## Number of chunks the rod gets broken into
dq = Q/N          ## Charge of a given chunk depends on the number of chunks

EScale = 1e-4     ## Scale factor for visualizing arrows

### Initialize the electric field & define observation location

The calculation that we are doing is the discrete (numerical) form of:

$$\vec{E}(\mathbf{r})=\dfrac{1}{4\pi\varepsilon_0}\int \dfrac{dq}{|\vec{\mathfrak{r}}|^2}\hat{\mathfrak{r}}$$

That is, we are doing a discrete sum over $N$ chunks,

$$\vec{E}(\mathbf{r})\approx\sum_i^N d\vec{E}_i= \dfrac{1}{4\pi\varepsilon_0}\sum_i^N \dfrac{dq_i}{|\vec{\mathfrak{r}}_i|^2}\hat{\mathfrak{r}}_i$$

To start this summation, we begin by initializing the net electric field to zero. This is a typical practice to ensure the numerical integral starts with the correct initial value as each contribution to the total field ($d\vec{E}$) will be added to this initial value (numerical superposition of the field).

Additionally, the observation location ($\mathbf{r}$) is the same for every term in the sum, so we define that as well.

In [3]:
Enet = vector(0,0,0)      ## Initialize the electric field
rObs = vector(0.1,0,0)    ## Set the observation location

## Performing the integral

The code below, when completed, will:

* Draw the rod and place white spheres at the centers of the chunks (it already does this, try running it)
* Perform the numerical integral (**your job for this problem**)
* Draw yellow arrows to show the contribution of each chunk (uncomment the line starting with ```dEArrow``` once you get the numerical integral working)
* Draw a white arrow to represent the net electric field at the observation location (it's already uncommented, but will show zero until you get the integration working).

The first few lines of code set up the canvas (just the visual window) and place the rod at the location we decided on. For each execution of the ```while``` loop,

* the source location is calculated (done for you),
* a white sphere representing that little chunk is created at the source location (done for you),
* the electric field contribution from that chunk is calculated (```dE```, **your job for this problem**),
* a yellow arrow is created to represent that chunk's contribution to the total electric field (uncomment that line once ``dE`` is calculated properly), and
* the chunk's contribution is added to the existing total field (**your job for this problem**)

Below the code is annotated a bit more.

In [11]:
## Creates the scene, which is just the visual window below
scene=canvas(title="Electric Field Due to a Uniformly Charged Rod")

## Creates a rod positioned as defined in the problem
rod = cylinder(pos=vector(0,-0.5,0), axis=vector(0,L,0), radius=0.01, color=color.red) 

## Initializes the counter
i = 0
## Intital the field to 0 for each run 
Enet = vector(0,0,0)  
## Calculation loop that places N charges and (when completed) calculates the contribution of 
## each charge to the total electric field, adding that contribution to the total
while i < N:
    
    ## Defines each source location as the center of the chunk
    rSource = vector(0,L/2-(i+0.5)*L/N,0)
    
    ## Places a sphere at the source location to allow visualization of the chunks
    sourceCharge = sphere(pos=vector(rSource), radius=0.02, color=color.white)
    
    ### Enter Your Physics Calculations Here for dE and Enet
    r = rObs - rSource
    dE = (k*dq/mag(r)**2)*(norm(r))
    Enet += dE
    
    ## Uncomment this line once you have determined dE
    dEArrow = arrow(pos=rObs, axis=EScale*dE, color=color.yellow)
    
    i+=1 ##Danny I love you but please just use +=
    
EnetArrow = arrow(pos=rObs, axis=EScale*Enet, color=color.white)
print("The net electric field at point P = ",Enet, "N/C")

<IPython.core.display.Javascript object>

The net electric field at point P =  <11740, 2.27374e-13, 0> N/C


In [16]:
##Calculate known value based on given formula 
E_true_x = k * Q * (1/(.1*sqrt(.1**2+(L/2)**2)))
E_true = vector(E_true_x,0,0)

i = 0
Enet = vector(0,0,0)  
N = 4 
error = 1000000000000000
## Calculation loop that places N charges and (when completed) calculates the contribution of 
## each charge to the total electric field, adding that contribution to the total
while error > .001:
    while i < N:
    
        ## Defines each source location as the center of the chunk
        rSource = vector(0,L/2-(i+0.5)*L/N,0)

        r = rObs - rSource
        dE = (k*dq/mag(r)**2)*(norm(r))
        Enet += dE
        i+=1
    
    error = abs(mag(Enet)-mag(E_true))/abs(mag(E_true))
    N+=1
    
print("The net electric field at point P = ",Enet, "N/C")
print("Iterations to .1\% Error: ", N)

i = 0
Enet = vector(0,0,0)  
N = 4 
error = 1000000000000000
## Calculation loop that places N charges and (when completed) calculates the contribution of 
## each charge to the total electric field, adding that contribution to the total
while error > .000001:
    while i < N:
    
        ## Defines each source location as the center of the chunk
        rSource = vector(0,L/2-(i+0.5)*L/N,0)

        r = rObs - rSource
        dE = (k*dq/mag(r)**2)*(norm(r))
        Enet += dE
        i+=1
    
    error = abs(mag(Enet)-mag(E_true))/abs(mag(E_true))
    N+=1
    
print("The net electric field at point P = ",Enet, "N/C")
print("Iterations to 5 Sigma Error: ", N)

##Accurate simulations are Incredibly computuationally expensive and not super practical. My little laptop cannot run this. 

KeyboardInterrupt: 